In [190]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Ruta del archivo CSV
ruta_mareapich= r'C:\Users\marib\OneDrive\Escritorio\295\reconstrucion_residuo_pich.csv'
# Carga del archivo de pichidangui generado en el código previo
mareapich= pd.read_csv(ruta_mareapich)
mareapich['fecha'] = pd.to_datetime(mareapich['fecha']) # Asegurarse del formato DateTime
# Carga de datos del Humedal de Pichicuy
ruta_csv_costab = r'C:\Users\marib\OneDrive\Escritorio\295\Costa_28July2023-18Aug2024 1(in).csv'
costab= pd.read_csv(ruta_csv_costab)
# Los datos de Pichicuy tienen algunos errores en la serie DateTime
# Corregir los datos de hora 00:00
N = 13
costab['DateTime'] = costab['DateTime'].apply(lambda x: x + ' 00:00' if len(x) < N else x)
costab['DateTime'] = pd.to_datetime(costab['DateTime'], dayfirst=True) #El formato de esta serie es día/hora, originalmente
# Corregir los datos de waterlevel, en relación al nivel promedio del mar.
costab['WL'] = costab['WL'] - 0.781234

In [191]:
# Filtrar el DataFrame para que incluya solo datos desde el 11 de enero de 2024 al 30 junio de 2024
start_date = '2024-01-01'
end_date = '2024-05-31'
#end_date = '2024-01-20' Para la visualización de datos se usa desde el 11 de enero hasta el 20 de enero.

# Aplicar el filtro
f_costab = costab[(costab['DateTime'] >= start_date) & (costab['DateTime'] <= end_date)]
f_mareapich = mareapich[(mareapich['fecha'] >= start_date) & (mareapich['fecha'] <= end_date)]

In [192]:
%matplotlib qt 
# Graficar los datos del archivo CSV
#plt.xlabel('Fecha', fontsize=20) # Label de x
plt.ylabel('Nivel del Mar [m]', fontsize=22) # Label de y
plt.plot(f_mareapich['fecha'], f_mareapich['Reconstruccion Pichidangui'], color='navy', label='Nivel de la Marea [m]', linewidth = 2) # Ploteo de datos
plt.plot(f_costab['DateTime'], f_costab['WL'], color='lightseagreen', label='Nivel de Agua Laguna [m]', linewidth = 5) # Ploteo de datos
plt.xticks(fontsize=22)                 
plt.yticks(fontsize=22) 

# Añadir leyenda
plt.tight_layout()
plt.legend(loc='upper right', fontsize=20)

# Mostrar el gráfico
plt.show() # Mostrar figura

In [196]:
# Filtrar el DataFrame para que incluya solo datos desde el 11 de enero de 2024 al 30 junio de 2024
start_date = '2024-01-11'
end_date = '2024-01-20'
#end_date = '2024-01-20' Para la visualización de datos se usa desde el 11 de enero hasta el 20 de enero.

# Aplicar el filtro
f_costab = costab[(costab['DateTime'] >= start_date) & (costab['DateTime'] <= end_date)]
f_mareapich = mareapich[(mareapich['fecha'] >= start_date) & (mareapich['fecha'] <= end_date)]

In [198]:
# Los datos de Pichicuy son muy ruidosos, mientras que los datos de marea gravitacional son suaves, por lo tanto suavizamos
# los datos.
from scipy.signal import savgol_filter
wl_pichicuy = f_costab['WL']
Smoot_wlpichicuy = savgol_filter(wl_pichicuy, 300, 3)
f_costab['WL_smooth'] = Smoot_wlpichicuy

C:\Users\marib\AppData\Local\Temp\ipykernel_12472\723283249.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_costab['WL_smooth'] = Smoot_wlpichicuy


## Implementación de la Ley de Darcy

In [267]:
slope_tan = 0.076 # Pendiente promedio de la playa. Calculado a partir de http://coastsat.wrl.unsw.edu.au/
L_w = 406.31 # Largo de la laguna. Medido en Google Earth
A_laguna = 11735.88 # Area de la laguna. Medido en Google Earth
L0 = 100 # Distancia de la laguna al mar cuando h_m = 0
h_m0 = 0 
K_1 = 10**-1 # Conductividad hidraúlica
K_3 = 10**-3 # Conductividad hidraúlica
K_6 = 10**-6 # Conductividad hidraúlica

# Cambio de la distancia al mar debido a subida y bajada de la marea en base a triginometría
# L -> distancia del Humedal a la costa
# h_m0 -> nivel de la marea relativo a L
# tan -> Pendiente de la playa
# data -> nombre de la serie de nivel de marea
# datetime -> nombre de la serie temporal
# time -> tiempo actual
def L_change(L, h_m0, tan, df, data, datetime, time):
    L_new = L - (df[df[datetime] == time][data] - h_m0)/tan
    #L_new = L
    return L_new

# Ley de Darcy para calcular el alto de la interfaz w
# L -> distancia del Humedal a la costa (calculado a partir de L_change)
# L_w -> Ancho de la laguna "Costa"
# K -> Conductividad hidraulica
# df1 -> dataframe de la laguna costa
# df2 -> dataframe de marea de Pichidangui
# data1 -> nombre de la serie de nivel de la laguna "Costa"
# data2 -> nombre de la serie de nivel de marea
# datetime1 -> nombre de la serie temporal de Pichicuy
# datetime2 -> nombre de la serie temporal de Pichidangui
# time -> tiempo actual
# prev_time -> tiempo anerior
def darcyslaw(L, L_w, K, df1, df2, data1, data2, datetime1, datetime2, time, prev_time):
    # Diferencia de altura entre la laguna "Costa" y el nivel de la marea
    delta_h = float(df1[df1[datetime1] == time][data1]) -  float(df2[df2[datetime2] == time][data2])
    # Paso temporal
    delta_t = time.timestamp() - prev_time.timestamp()
    # Eliminación de casos cercanos a la divergencia dh = 0
    if abs(delta_h) < (2.5*10**-1)* float(df1[df1[datetime1] == time][data1]):
        Q = float('nan')
        w = float('nan')
    else:
        # Cálculo del caudal en base al cambio de nivel de agua de la laguna "Costa"
        Q = (float(df1[df1[datetime1] == time][data1]) - float(df1[df1[datetime1] == prev_time][data1]))*A_laguna/(delta_t)
        # Cálculo de la altura de la interfaz
        w = L*Q/(L_w*K*delta_h)
    # Entrega los resultados en valor absoluto
    return abs(w), Q

In [269]:
# Encontrar datos de fecha comunes
common_datetimes = pd.Series(list(set(f_costab['DateTime']).intersection(set(f_mareapich['fecha']))))

# Filtrar ambos dataframes para que tengan los mismos datos de fecha
darcy_costab = f_costab[f_costab['DateTime'].isin(common_datetimes)].reset_index(drop=True)
darcy_mareapich = f_mareapich[f_mareapich['fecha'].isin(common_datetimes)].reset_index(drop=True)
print(len(darcy_costab))

11459


In [273]:
# Creacion de vectores de alto w y caudal Q
w_smooth_1 = []
w_smooth_3 = []
w_smooth_6 = []
Q_smooth = []

# Iteración a lo largo de la serie temporal
for i in range(1, len(darcy_costab)):
    current_time = darcy_costab.loc[i, 'DateTime']       # Tiempo actual
    previous_time = darcy_costab.loc[i - 1, 'DateTime']  # Tiempo previo
    L = L_change(L0, h_m0, slope_tan, darcy_mareapich, 'Reconstruccion Pichidangui', 'fecha', current_time)
    # Aturas para distintos K
    width_smooth_1, Flow_smooth = darcyslaw(float(L), L_w, K_1, darcy_costab, darcy_mareapich, 'WL_smooth', 'Reconstruccion Pichidangui', 'DateTime', 'fecha', current_time, previous_time)
    width_smooth_3, Flow_smooth = darcyslaw(float(L), L_w, K_3, darcy_costab, darcy_mareapich, 'WL_smooth', 'Reconstruccion Pichidangui', 'DateTime', 'fecha', current_time, previous_time)
    width_smooth_6, Flow_smooth = darcyslaw(float(L), L_w, K_6, darcy_costab, darcy_mareapich, 'WL_smooth', 'Reconstruccion Pichidangui', 'DateTime', 'fecha', current_time, previous_time)
    w_smooth_1.append(width_smooth_1)
    w_smooth_3.append(width_smooth_3)
    w_smooth_6.append(width_smooth_6)
    Q_smooth.append(Flow_smooth)


C:\Users\marib\AppData\Local\Temp\ipykernel_12472\2213484679.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  width_smooth_1, Flow_smooth = darcyslaw(float(L), L_w, K_1, darcy_costab, darcy_mareapich, 'WL_smooth', 'Reconstruccion Pichidangui', 'DateTime', 'fecha', current_time, previous_time)
C:\Users\marib\AppData\Local\Temp\ipykernel_12472\2958514393.py:36: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  delta_h = float(df1[df1[datetime1] == time][data1]) -  float(df2[df2[datetime2] == time][data2])
C:\Users\marib\AppData\Local\Temp\ipykernel_12472\2958514393.py:40: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if abs(delta_h) < (2.5*10**-1)* float(df1[df1[datetime1] == time][data1]):
C:\Us

In [275]:
# Insertamos un elemento al principio de cada vector para igualar la cantidad de datos
w_smooth_1.insert(0, 0)
w_smooth_3.insert(0, 0)
w_smooth_6.insert(0, 0)
Q_smooth.insert(0, 0)
#w_real.insert(0, 0)

### Es importante saber cuando el nivel del agua de la laguna "Costa" esta por sobre el nivel de la marea

In [278]:
# Creamos un dataframe con los niveles de agua de la laguna "Costa" y de marea, además del tiempo y una designación
# del tipo (por sobre, igual o bajo el nivel de la marea)
df_new = pd.DataFrame({
    'DateTime': darcy_costab['DateTime'],
    'WL_pichicuy': darcy_costab['WL_smooth'],
    'WL_pichidangui': darcy_mareapich['Reconstruccion Pichidangui'],
    'Type': ['None']*len(darcy_costab)
})
# Itereamos sobre el dataframe para catalogar
for i in range(0, len(df_new)):
    row = df_new.loc[i]
    if i!=0:
        row_prev = df_new.loc[i-1]
        # Si cruza el 0 asignar Equal
        if (float(row['WL_pichicuy']) - float(row['WL_pichidangui']))>0 and (float(row_prev['WL_pichicuy']) - float(row_prev['WL_pichidangui']))<0:
            df_new.loc[i, 'Type'] = 'Equal'
        # Si cruza el 0 asignar Equal
        elif (float(row['WL_pichicuy']) - float(row['WL_pichidangui']))<0 and (float(row_prev['WL_pichicuy']) - float(row_prev['WL_pichidangui']))>0:
            df_new.loc[i, 'Type'] = 'Equal'
        # Si la laguna Costa esta por sobre la marea asignar Greater
        elif float(row['WL_pichicuy']) > float(row['WL_pichidangui']):
            df_new.loc[i, 'Type'] = 'Greater' 
        # Si la laguna Costa esta bajo la marea asignar Lesser
        elif float(row['WL_pichicuy']) < float(row['WL_pichidangui']):
            df_new.loc[i, 'Type'] = 'Lesser'
        # Si ambos niveles son iguales asignar Equal
        elif (float(row['WL_pichicuy']) == float(row['WL_pichidangui'])):
            df_new.loc[i, 'Type'] = 'Equal'
    else:
        # Si la laguna Costa esta por sobre la marea asignar Greater
        if float(row['WL_pichicuy']) > float(row['WL_pichidangui']):
            df_new.loc[i, 'Type'] = 'Greater' 
        # Si la laguna Costa esta bajo la marea asignar Lesser
        elif float(row['WL_pichicuy']) < float(row['WL_pichidangui']):
            df_new.loc[i, 'Type'] = 'Lesser'
# Checkeamos que no falten datos catalogados
print(df_new[df_new['Type'] == 'None'])
df_greater = df_new[df_new['Type'] == 'Greater']
df_lesser = df_new[df_new['Type'] == 'Lesser']
df_equal = df_new[df_new['Type'] == 'Equal']

Empty DataFrame
Columns: [DateTime, WL_pichicuy, WL_pichidangui, Type]
Index: []


In [288]:
%matplotlib qt 

# Ploteamos los resultados de w para K = 10e-3

fig, ax2 = plt.subplots()

ax2.scatter(df_greater['DateTime'], df_greater['WL_pichicuy'], color='lightseagreen', s = 11, label = 'Laguna - Sobre el NM')
ax2.set_ylabel('Nivel de Agua Laguna [m]', fontsize = 22)
ax2.set_ylim(0.4, 0.47)
ax2.tick_params(axis='x', labelsize=16)  
ax2.tick_params(axis='y', labelsize=22)  
ax3 = ax2.twinx()
ax3.scatter(df_lesser['DateTime'], df_lesser['WL_pichicuy'], color='slateblue', s = 11, label = 'Laguna - Bajo el NM')
#ax3.set_ylabel('Y2 scale', color='lightseagreen')
ax3.set_ylim(0.4, 0.47)
ax3.tick_params(axis='y', labelsize=0)  
ax4 = ax2.twinx()
ax4.scatter(df_equal['DateTime'], df_equal['WL_pichicuy'], color='black', s = 50, label = 'Laguna - Igual al NM')
#ax4.set_ylabel('Y2 scale', color='lightseagreen')
ax4.set_ylim(0.4, 0.47)
ax4.tick_params(axis='y', labelsize=0)  
# Third curve on a third y-axis
ax5 = ax2.twinx()
#ax5.spines['right'].set_position(('outward', 90))
ax5.plot(darcy_costab['DateTime'], w_smooth, color='yellowgreen', linestyle = '-', linewidth = 1.5, label = 'Altura de la interfaz')
ax5.set_ylabel('Altura de la interfaz [m]', fontsize = 22)
ax5.set_ylim(0, 0.15)
ax5.tick_params(axis='y', labelsize=22)  
#fig.legend(fontsize = 15, loc = 'upper left', framealpha = 1)
plt.show()

In [402]:
# Ploteamos los resultados de w para K = 10e-1, 10e-3 y 10e-6

fig, ax2 = plt.subplots()

ax2.plot(df_new['DateTime'], df_new['WL_pichicuy'], color='slategray', linewidth = 5.5, label = 'Nivel de la Laguna')
ax2.set_ylabel('Nivel de Agua Laguna [m]', fontsize = 22)
ax2.set_ylim(0.4, 0.47)
ax2.tick_params(axis='x', labelsize=16)  
ax2.tick_params(axis='y', labelsize=22)  
# Third curve on a third y-axis
ax5 = ax2.twinx()
#ax5.spines['right'].set_position(('outward', 90))
ax5.plot(darcy_costab['DateTime'], w_smooth_1, color='lightseagreen', linestyle = '-', linewidth = 2, label = 'Altura de la interfaz para $K = 10^{-1} ms^{-1}$')
ax5.set_ylim(0, 0.15)
ax5.tick_params(axis='y', labelsize=22, colors = 'lightseagreen')  
ax7 = ax2.twinx()
ax7.spines['right'].set_position(('outward', 70))
ax7.plot(darcy_costab['DateTime'], w_smooth_3, color='yellowgreen', linestyle = '-', linewidth = 3, label = 'Altura de la interfaz para $K = 10^{-3} ms^{-1}$')
ax7.set_ylim(0, 7)
ax7.tick_params(axis='y', labelsize=22, colors = 'yellowgreen')  
ax6 = ax2.twinx()
ax6.spines['right'].set_position(('outward', 100))
ax6.plot(darcy_costab['DateTime'], w_smooth_6, color='teal', linestyle = '-', linewidth = 2, label = 'Altura de la interfaz para $K = 10^{-6} ms^{-1}$')
ax6.set_ylim(0, 4000)
ax6.tick_params(axis='y', labelsize=22, colors = 'teal')  
ax6.set_ylabel('Altura de la interfaz [m]', fontsize = 22)
#fig.legend(fontsize = 20, loc = 'upper left', framealpha = 1)
plt.show()

In [292]:
# Ploteamos los resultados de Q

fig, ax2 = plt.subplots()

ax2.scatter(df_greater['DateTime'], df_greater['WL_pichicuy'], color='lightseagreen', s = 11, label = 'Laguna - Sobre el NM')
ax2.set_ylabel('Nivel de Agua Laguna [m]', fontsize = 22)
ax2.set_ylim(0.4, 0.47)
ax2.tick_params(axis='x', labelsize=16)  
ax2.tick_params(axis='y', labelsize=22)  
ax3 = ax2.twinx()
ax3.scatter(df_lesser['DateTime'], df_lesser['WL_pichicuy'], color='slateblue', s = 11, label = 'Laguna - Bajo el NM')
#ax3.set_ylabel('Y2 scale', color='lightseagreen')
ax3.set_ylim(0.4, 0.47)
ax3.tick_params(axis='y', labelsize=0)  
ax4 = ax2.twinx()
ax4.scatter(df_equal['DateTime'], df_equal['WL_pichicuy'], color='black', s = 50, label = 'Laguna - Igual al NM')
#ax4.set_ylabel('Y2 scale', color='lightseagreen')
ax4.set_ylim(0.4, 0.47)
ax4.tick_params(axis='y', labelsize=0)  
# Third curve on a third y-axis
ax5 = ax2.twinx()
#ax5.spines['right'].set_position(('outward', 90))
ax5.plot(darcy_costab['DateTime'], Q_smooth, color='indianred', linestyle = '-', linewidth = 1.5, label = 'Caudal')
ax5.set_ylabel('Caudal $[m^{3}s^{-1}]$', fontsize = 22)
#ax5.set_ylim(0, 7.5)
ax5.tick_params(axis='y', labelsize=22)  
#fig.legend(fontsize = 15, loc = 'upper left', framealpha = 1)
plt.axhline(y=0, color='black', linestyle='--')
plt.show()